<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/tf2-12-5-rnn_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


# Lab


In [24]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Min-Max 스케일러
# -------------------------
def MinMaxScaler(data):
    numerator = data - np.min(data, axis=0)
    denominator = np.max(data, axis=0) - np.min(data, axis=0)
    return numerator / (denominator + 1e-7)

# -------------------------
# 하이퍼파라미터
# -------------------------
seq_length = 7
data_dim = 5
output_dim = 1
learning_rate = 0.01
iterations = 500

# -------------------------
# 데이터 로드
# -------------------------
xy = np.loadtxt('../data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1]

# Train/Test 분리
train_size = int(len(xy) * 0.7)
train_set = MinMaxScaler(xy[0:train_size])
test_set = MinMaxScaler(xy[train_size - seq_length:])

# -------------------------
# 데이터셋 생성
# -------------------------
def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(0, len(time_series) - seq_length):
        x = time_series[i:i + seq_length, :]
        y = time_series[i + seq_length, [-1]]  # Close 값
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print("Train:", trainX.shape, trainY.shape)
print("Test:", testX.shape, testY.shape)

# -------------------------
# 모델 정의
# -------------------------
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=10, input_shape=(seq_length, data_dim)),
    tf.keras.layers.Dense(units=output_dim)
])

model.summary()
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

# -------------------------
# 학습
# -------------------------
model.fit(trainX, trainY, epochs=iterations, verbose=1)

# -------------------------
# 예측
# -------------------------
test_predict = model.predict(testX)

# -------------------------
# 시각화
# -------------------------
plt.figure(figsize=(10,5))
plt.plot(testY, label='Actual Price')
plt.plot(test_predict, label='Predicted Price')
plt.legend()
plt.xlabel("Time Period")
plt.ylabel("Stock Price")
plt.title("Stock Price Prediction with LSTM")
plt.show()


FileNotFoundError: ../data-02-stock_daily.csv not found.